In [1]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras import optimizers
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
import matplotlib.pyplot as plt
import json
from PIL import Image
from IPython.display import Image as disp_image
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt



In [ ]:
img_height = 256
img_width = 256
channels = 3
batch_size = 8
epochs = 20
train_datagen = ImageDataGenerator(rotation_range=40,
 width_shift_range=0.2,
 height_shift_range=0.2,
 rescale=1./255,
 shear_range=0.2,
 zoom_range=0.2,
 #horizontal_flip=True,
fill_mode='nearest',
 validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)
training_set = train_datagen.flow_from_directory(
 './dataset/training',
 target_size = (img_height, img_width),
 batch_size = batch_size,
 class_mode = 'binary',
 subset = 'training',
 shuffle=True)
validation_set = train_datagen.flow_from_directory(
 './dataset/training',
 target_size = (img_height, img_width),
 batch_size = batch_size,
 class_mode = 'binary',
 subset = 'validation',
 shuffle=False)
test_set = train_datagen.flow_from_directory(
 './dataset/test',
 target_size = (img_height, img_width),
 batch_size = 1,
 shuffle = False,
 class_mode = 'binary')
print(training_set.class_indices)


In [ ]:
# Covid image example
img_files = os.listdir('dataset/training/covid')
img_path = img_files[np.random.randint(0,len(img_files))]
img = Image.open('dataset/training/covid/{}'.format(img_path))
img.thumbnail((500, 500))
display(img)

In [ ]:
# Healthy image example
img_files = os.listdir('dataset/training/healthy')
img_path = img_files[np.random.randint(0,len(img_files))]
img = Image.open('dataset/training/healthy/{}'.format(img_path))
img.thumbnail((500, 500))
display(img)


In [ ]:
# VGG16
model = VGG16(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, channels))
for layer in model.layers[:-5]:
 layer.trainable = False
top_model = Sequential()
top_model.add(model)
top_model.add(Flatten())
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))
print(model.summary())
print(top_model.summary())

In [ ]:
# Compile Model
top_model.compile(loss='binary_crossentropy',
 optimizer=optimizers.RMSprop(lr=1e-4, decay=1e-6),
 metrics=['accuracy'])
# Save Model
history = top_model.fit_generator(
 training_set,
 steps_per_epoch=training_set.n // batch_size,
 epochs=epochs,
 validation_data=validation_set,
 validation_steps=validation_set.n // batch_size)
# Save Model
top_model.save('output/covid_model.h5', save_format='h5')

In [ ]:
# Plot Accuracy and Loss
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))
plt.plot(epochs, acc, 'b', label='Training Accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'b', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
# Model Performance on test set
test_pred = top_model.evaluate_generator(test_set,
 steps=test_set.n//batch_size,
 use_multiprocessing=False,
 verbose=1)
print('Test loss: ', test_pred[0])
print('Test accuracy: ', test_pred[1])


In [ ]:
test_pred = top_model.predict(test_set,
 steps=test_set.n,
 use_multiprocessing=False,
 verbose=1)
test_pred_class = (test_pred >= 0.5)*1
# Confusion Matrix
cm = confusion_matrix(test_set.classes,
 test_pred_class)
ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax); #annot=True to annotate cells
# labels, title and ticks
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels');
ax.set_title('Confusion Matrix');
ax.xaxis.set_ticklabels(['COVID-19', 'Healthy']); ax.yaxis.set_ticklabels(['COVID-19', 'Healthy']);

In [ ]:
# Classification Report
print(classification_report(test_set.classes,
 test_pred_class))

In [ ]:
img_height = 256
img_width = 256
channels = 3
batch_size = 8
epochs = 20
train_datagen = ImageDataGenerator(rotation_range=40,
 width_shift_range=0.2,
 height_shift_range=0.2,
 rescale=1./255,
 shear_range=0.2,
 zoom_range=0.2,
 #horizontal_flip=True,
fill_mode='nearest',
 validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)
training_set = train_datagen.flow_from_directory('./multiclass_dataset/training',
 target_size = (img_height, img_width),
 batch_size = batch_size,
 class_mode = 'categorical',
 subset = 'training',
 shuffle=True)
validation_set = train_datagen.flow_from_directory('./multiclass_dataset/training',
 target_size = (img_height, img_width),
 batch_size = batch_size,
 class_mode = 'categorical',
 subset = 'validation',
 shuffle=False)
test_set = train_datagen.flow_from_directory('./multiclass_dataset/test',
 target_size = (img_height, img_width),
 batch_size = 1,
 shuffle = False,
 class_mode = 'categorical')
print(training_set.class_indices)

In [ ]:
# VGG16
model = VGG16(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, channels))
for layer in model.layers[:-5]:
 layer.trainable = False
top_model = Sequential()
top_model.add(model)
top_model.add(Flatten())
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(len(training_set.class_indices), activation='softmax'))
print(model.summary())
print(top_model.summary())

In [ ]:
# Compile Model
top_model.compile(loss='categorical_crossentropy',
 optimizer=optimizers.RMSprop(lr=1e-4, decay=1e-6),
 metrics=['accuracy'])
save_best = ModelCheckpoint('output/covid_pneumonia_model.h5', monitor='val_loss', verbose=0, save_best_only=True)
# Train Model
history = top_model.fit_generator(
 training_set,
 steps_per_epoch=training_set.n // batch_size,
 epochs=epochs,
 validation_data=validation_set,
 validation_steps=validation_set.n // batch_size,
 callbacks=[save_best])
# Save Model
top_model.save('output/covid_pneumonia_model.h5', save_format='h5')

In [ ]:
# Plot Accuracy and Loss
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))
plt.plot(epochs, acc, 'b', label='Training Accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'b', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
test_pred = top_model.predict(test_set,
 steps=test_set.n,
 use_multiprocessing=False,
 verbose=1)
test_pred_class = np.argsort(test_pred)[:,2]
# Confusion Matrix
cm = confusion_matrix(test_set.classes,
 test_pred_class)
ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax); #annot=True to annotate cells
# labels, title and ticks
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels');
ax.set_title('Confusion Matrix');
ax.xaxis.set_ticklabels(['COVID-19', 'Healthy', 'Pneumonia']); ax.yaxis.set_ticklabels(['COVID-19', 'Healthy', 'Pneumonia']);

In [ ]:
# Classification Report
print(classification_report(test_set.classes,
 test_pred_class))

In [ ]:
# Accuracy
accuracy_score(test_set.classes,
 test_pred_class)